In [52]:
import pandas as pd
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from collections import Counter, OrderedDict

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

In [19]:
file_location = '/Users/hyunoochang/code/chyunoo/wineteller/raw_data/winemag-data_first150k.csv'

data = pd.read_csv(file_location, index_col='Unnamed: 0')
print(data.shape)

(150930, 10)


In [20]:
data.head(5)

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [21]:
labels = data['variety']
descriptions = data['description']

In [22]:
print('{}   :   {}'.format(labels.tolist()[0], descriptions.tolist()[0]))
print('{}   :   {}'.format(labels.tolist()[1], descriptions.tolist()[1]))
print('{}   :   {}'.format(labels.tolist()[2], descriptions.tolist()[2]))

Cabernet Sauvignon   :   This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022–2030.
Tinta de Toro   :   Ripe aromas of fig, blackberry and cassis are softened and sweetened by a slathering of oaky chocolate and vanilla. This is full, layered, intense and cushioned on the palate, with rich flavors of chocolaty black fruits and baking spices. A toasty, everlasting finish is heady but ideally balanced. Drink through 2023.
Sauvignon Blanc   :   Mac Watson honors the memory of a wine once made by his mother in this tremendously delicious, balanced and complex botrytised white. Dark gold in color, it layers toasted hazelnut, pear compote and orange peel flavors, reveling in the succulence of its 122 

In [23]:
varietal_counts = labels.value_counts()
print(varietal_counts[:50])

Chardonnay                       14482
Pinot Noir                       14291
Cabernet Sauvignon               12800
Red Blend                        10062
Bordeaux-style Red Blend          7347
Sauvignon Blanc                   6320
Syrah                             5825
Riesling                          5524
Merlot                            5070
Zinfandel                         3799
Sangiovese                        3345
Malbec                            3208
White Blend                       2824
Rosé                              2817
Tempranillo                       2556
Nebbiolo                          2241
Portuguese Red                    2216
Sparkling Blend                   2004
Shiraz                            1970
Corvina, Rondinella, Molinara     1682
Rhône-style Red Blend             1505
Pinot Gris                        1365
Barbera                           1365
Cabernet Franc                    1363
Sangiovese Grosso                 1346
Pinot Grigio             

In [31]:
print(varietal_counts[-10:])

Cococciola             1
Garnacha Tintorera     1
Malbec-Petit Verdot    1
Albarossa              1
Aidani                 1
Carignan-Syrah         1
Premsal                1
Muskat                 1
Syrah-Carignan         1
Carnelian              1
Name: variety, dtype: int64


In [38]:
test = data[:1000]

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude
...,...,...,...,...,...,...,...,...,...,...
995,Portugal,"This is a dry sparkling wine with tannins, spi...",Plexus Tinto,85,7.0,Tejo,NaN,NaN,Alicante Bouschet,Adega Cooperativa do Cartaxo
996,Portugal,"This is a soft, creamy wine with fruitiness an...",Blanc de Blancs Bruto,85,12.0,Beira Atlantico,NaN,NaN,Portuguese Sparkling,Adega de Cantanhede
997,France,This is a soft wine whose herbal character is ...,La Galope,85,15.0,Southwest France,Côtes de Gascogne,NaN,Sauvignon Blanc,Domaine de l'Herré
998,France,This white wine is fruity and crisp with a del...,NaN,85,9.0,Southwest France,Côtes de Gascogne,NaN,White Blend,Domaine du Touja


In [39]:
reviews_list = list(test['description'])
reviews_list = [str(r) for r in reviews_list]
full_corpus = ' '.join(reviews_list)
sentences_tokenized = sent_tokenize(full_corpus)

print(sentences_tokenized[:5])

['This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak.', 'Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background.', 'Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance.', 'Enjoy 2022–2030.', 'Ripe aromas of fig, blackberry and cassis are softened and sweetened by a slathering of oaky chocolate and vanilla.']


In [42]:
stop_words = set(stopwords.words('english')) 

punctuation_table = str.maketrans({key: None for key in string.punctuation})
sno = SnowballStemmer('english')

def normalize_text(raw_text):
    try:
        word_list = word_tokenize(raw_text)
        normalized_sentence = []
        for w in word_list:
            try:
                w = str(w)
                lower_case_word = str.lower(w)
                stemmed_word = sno.stem(lower_case_word)
                no_punctuation = stemmed_word.translate(punctuation_table)
                if len(no_punctuation) > 1 and no_punctuation not in stop_words:
                    normalized_sentence.append(no_punctuation)
            except:
                continue
        return normalized_sentence
    except:
        return ''

# sentence_sample = sentences_tokenized[:10]
normalized_sentences = []
for s in sentences_tokenized:
    normalized_text = normalize_text(s)
    normalized_sentences.append(normalized_text)

In [46]:
normalized_sentences[:3]

[['tremend',
  '100',
  'variet',
  'wine',
  'hail',
  'oakvill',
  'age',
  'three',
  'year',
  'oak'],
 ['juici',
  'redcherri',
  'fruit',
  'compel',
  'hint',
  'caramel',
  'greet',
  'palat',
  'frame',
  'eleg',
  'fine',
  'tannin',
  'subtl',
  'minti',
  'tone',
  'background'],
 ['balanc', 'reward', 'start', 'finish', 'year', 'ahead', 'develop', 'nuanc']]

In [47]:
phrases = Phrases(normalized_sentences)
phrases = Phrases(phrases[normalized_sentences])

ngrams = Phraser(phrases)

phrased_sentences = []
for sent in normalized_sentences:
    phrased_sentence = ngrams[sent]
    phrased_sentences.append(phrased_sentence)

full_list_words = [item for sublist in phrased_sentences for item in sublist]

In [51]:
full_list_words[:10]

['tremend',
 '100_variet',
 'wine',
 'hail',
 'oakvill',
 'age',
 'three',
 'year',
 'oak',
 'juici']

In [54]:
word_counts = Counter(full_list_words)
sorted_counts = OrderedDict(word_counts.most_common(5000))
counter_df = pd.DataFrame.from_dict(sorted_counts, orient='index')
top_5000_words = counter_df.head(5000)
counter_df.to_csv('top_5000_descriptors.csv')

In [55]:
top_5000_words

,0
wine,637
flavor,537
fruit,386
aroma,328
palat,298
...,...
arinto,1
maria,1
gome,1
check,1


In [58]:
map_location = '/Users/hyunoochang/code/chyunoo/wineteller/raw_data/descriptor_mapping.csv'

In [61]:
descriptor_mapping = pd.read_csv(map_location)
descriptor_mapping

,raw descriptor,level_3,level_2,level_1
0,abras,abrasive,high_tannin,tannin
1,acacia,acacia,flowery,flower
2,acacia_flower,acacia,flowery,flower
3,aciddriven,acid_driven,high_acid,acid
4,aggress,aggressive,high_acid,acid
...,...,...,...,...
1010,zest,zest,citrus_fruit,fruit
1011,zesti,zesty,high_acid,acid
1012,zing,zingy,high_acid,acid
1013,zingi,zingy,high_acid,acid


In [73]:
descriptor_list = ['body', 'finish', 'complexity']

filtered_descriptor_mapping = descriptor_mapping[descriptor_mapping['level_1'].isin(descriptor_list)]

In [85]:
filtered_descriptor_mapping.groupby("level_1").nunique()

,raw descriptor,level_3,level_2
level_1,,,
body,57,34,3
complexity,16,14,2
finish,6,3,2


In [98]:
list(filtered_descriptor_mapping["raw descriptor"])

['airi',
 'big_bold',
 'bullish',
 'blocki',
 'chunki',
 'solid_chunki',
 'clampi',
 'close',
 'complex',
 'complic',
 'dainti',
 'deep',
 'depth',
 'eas',
 'easi_sip',
 'eleg',
 'expans',
 'extract',
 'feminin',
 'finess',
 'ampl_weight',
 'bold',
 'full_bodi',
 'fullbodi',
 'soupi',
 'weighti',
 'hearti',
 'heavi',
 'heavier',
 'heavyweight',
 'lavish',
 'rather_lean',
 'length',
 'lengthi',
 'long_mouthwat',
 'longlast',
 'lightest',
 'light',
 'light_bodi',
 'lightbodi',
 'lighter',
 'lighter_style',
 'lightfoot',
 'lightweight',
 'thin',
 'linear',
 'straightforward',
 'lush',
 'luxuri',
 'mass',
 'medium_bodi',
 'medium_fullbodi',
 'medium_weight',
 'mediumbodi',
 'mediumweight',
 'medium_length',
 'mediumlength_finish',
 'modest',
 'onedimension',
 'opul',
 'plump',
 'quaffer',
 'refin',
 'rich',
 'richer',
 'robust',
 'uncompl',
 'stout',
 'sturdi',
 'succul',
 'superrich',
 'syrup',
 'syrupi',
 'thick',
 'unoak',
 'viscos',
 'viscous',
 'voluptu',
 'weight']

In [86]:
def return_mapped_descriptor(word):
    if word in list(filtered_descriptor_mapping.index):
        normalized_word = filtered_descriptor_mapping['level_3'][word]
        return normalized_word
    else:
        return word

normalized_sentences = []
for sent in phrased_sentences:
    normalized_sentence = []
    for word in sent:
        normalized_word = return_mapped_descriptor(word)
        normalized_sentence.append(str(normalized_word))
    normalized_sentences.append(normalized_sentence)

In [88]:
normalized_sentences[:10]

[['tremend',
  '100_variet',
  'wine',
  'hail',
  'oakvill',
  'age',
  'three',
  'year',
  'oak'],
 ['juici',
  'redcherri',
  'fruit',
  'compel',
  'hint',
  'caramel',
  'greet',
  'palat',
  'frame',
  'eleg',
  'fine',
  'tannin',
  'subtl',
  'minti',
  'tone',
  'background'],
 ['balanc', 'reward', 'start', 'finish', 'year', 'ahead', 'develop', 'nuanc'],
 ['enjoy', '2022–2030'],
 ['ripe',
  'aroma',
  'fig',
  'blackberri_cassi',
  'soften',
  'sweeten',
  'slather',
  'oaki',
  'chocol',
  'vanilla'],
 ['full',
  'layer',
  'intens',
  'cushion',
  'palat',
  'rich',
  'flavor',
  'chocolati',
  'black_fruit',
  'bake_spice'],
 ['toasti', 'everlast', 'finish', 'headi', 'ideal', 'balanc'],
 ['drink', '2023'],
 ['mac',
  'watson',
  'honor',
  'memori',
  'wine',
  'onc',
  'made',
  'mother',
  'tremend',
  'delici',
  'balanc',
  'complex',
  'botrytis',
  'white'],
 ['dark',
  'gold',
  'color',
  'layer',
  'toast',
  'hazelnut',
  'pear',
  'compot',
  'orang_peel',
  'fl